In [ ]:
import os 
import pandas as pd
import geopandas as gpd
from energyemissionsregio.config import DATA_PATH, SHP_PATH, units
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.colors import SymLogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmcrameri import cm

In [ ]:
cwd = os.getcwd()

In [ ]:
lau_shp = gpd.read_file(os.path.join(SHP_PATH, "LAU.shp"))
lau_shp = lau_shp[lau_shp["code"].str.startswith(("DE", "ES"))]

de_lau_shp = lau_shp[lau_shp["code"].str.startswith("DE")]
es_lau_shp = lau_shp[lau_shp["code"].str.startswith("ES")]

#### Emissions from Food, beverages, and tobacco industries

In [ ]:
target_var = "ghg_emissions_from_fc_in_food_beverages_and_tobacco_industries"

In [ ]:
disagg_data = pd.read_csv(os.path.join(cwd, "..", "..", "data", "disaggregated_data", "GHG", f"{target_var}.csv"))

# convert Mt to tonnes so the values are seen in the map
disagg_data["value"] = disagg_data["value"] * 1e6

proxy_data_de = pd.read_csv(os.path.join(cwd, "..", "..", "data", "disaggregated_data", "de_employment_in_food_and_beverage_manufacturing.csv"))

proxy_data_es = pd.read_csv(os.path.join(cwd, "..", "..", "data", "disaggregated_data", "employment_in_manufacturing.csv")) 
proxy_data_es = proxy_data_es[proxy_data_es["region_code"].str.startswith("ES")].copy()

proxy_var_unit = "number"
disagg_var_unit = "tonnes" 

fig_path = os.path.join(cwd, "..", "..", "figures", "validation", f"{target_var}.png")
    
merged_proxy_data_de = pd.merge(
        de_lau_shp, proxy_data_de, left_on="code", right_on="region_code", how="left"
    )

merged_proxy_data_es = pd.merge(
        es_lau_shp, proxy_data_es, left_on="code", right_on="region_code", how="left"
    )

merged_disagg_data_de = pd.merge(
    de_lau_shp, disagg_data, left_on="code", right_on="region_code", how="left"
)

merged_disagg_data_es = pd.merge(
    es_lau_shp, disagg_data, left_on="code", right_on="region_code", how="left"
)

fig = plt.figure(figsize=(13, 13))
gs = fig.add_gridspec(2, 2, wspace=0.1, hspace=0)

# Germany - proxy --------

vmin, vmax = min(merged_proxy_data_de["value"]), max(merged_proxy_data_de["value"])
norm = SymLogNorm(linthresh=0.1, vmin=vmin, vmax=vmax)

ax1 = plt.subplot(gs[:1, :1])

merged_proxy_data_de.plot(
    column="value",
    cmap=cm.batlow_r,
    linewidth=0.8,
    ax=ax1,
    edgecolor="none",
    norm=norm,
)
# Hide spines (the borders of the plot)
ax1.spines["top"].set_visible(False)
ax1.spines["right"].set_visible(False)
ax1.spines["left"].set_visible(False)
ax1.spines["bottom"].set_visible(False)

# Hide ticks and tick labels
ax1.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

ax1.set_ylabel("Proxy values", fontsize=15)
ax1.set_title("Employment in food \nand beverage manufacturing")

# legend
divider = make_axes_locatable(ax1)
cax = divider.append_axes("right", size="3%", pad=0.1)
sm = plt.cm.ScalarMappable(cmap=cm.batlow_r, norm=norm)
sm._A = []
clb = fig.colorbar(sm, cax=cax, shrink=0.3)

clb.ax.tick_params(labelsize=10)
clb.ax.set_title(f"({proxy_var_unit})", fontsize=10)

# Spain - Proxy  --------
ax2 = plt.subplot(gs[:1, 1:])

merged_proxy_data_es.plot(
    column="value",
    cmap=cm.batlow_r,
    linewidth=0.8,
    ax=ax2,
    edgecolor="none",
    norm=norm,
)
ax2.axis("off")
ax2.set_title("Employment in manufacturing")

# legend
divider = make_axes_locatable(ax2)
cax = divider.append_axes("right", size="3%", pad=0.1)
sm = plt.cm.ScalarMappable(cmap=cm.batlow_r, norm=norm)
sm._A = []
clb = fig.colorbar(sm, cax=cax, shrink=0.3)

clb.ax.tick_params(labelsize=10)
clb.ax.set_title(f"({proxy_var_unit})", fontsize=10)

# Germany - Disagg --------

vmin, vmax = min(merged_disagg_data_de["value"]), max(merged_disagg_data_de["value"])
norm = SymLogNorm(linthresh=0.1, vmin=vmin, vmax=vmax)

ax3 = plt.subplot(gs[1:, :1])

merged_disagg_data_de.plot(
    column="value",
    cmap=cm.batlow_r,
    linewidth=0.8,
    ax=ax3,
    edgecolor="none",
    norm=norm,
)
# Hide spines (the borders of the plot)
ax3.spines["top"].set_visible(False)
ax3.spines["right"].set_visible(False)
ax3.spines["left"].set_visible(False)
ax3.spines["bottom"].set_visible(False)

# Hide ticks and tick labels
ax3.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

ax3.set_ylabel("Disaggregated values", fontsize=15)

# Spain - Disagg --------
ax4 = plt.subplot(gs[1:, 1:])

merged_disagg_data_es.plot(
    column="value",
    cmap=cm.batlow_r, 
    linewidth=0.8,
    ax=ax4,
    edgecolor="none",
    norm=norm,
)
ax4.axis("off")

# legend
divider = make_axes_locatable(ax4)
cax = divider.append_axes("right", size="3%", pad=0.1)
sm = plt.cm.ScalarMappable(cmap=cm.batlow_r, norm=norm)
sm._A = []
clb = fig.colorbar(sm, cax=cax, shrink=0.3)

clb.ax.tick_params(labelsize=10)
clb.ax.set_title(f"({disagg_var_unit})", fontsize=10)

fig_path = os.path.join("..", "..", "figures", 
                        "disaggregation_validation", 
                        "non_matching_subsector.png")

plt.savefig(fig_path, format="png", bbox_inches="tight", dpi=200)